<!--start-block-->
<hr style="height: 1px;">
<i>This code was authored by the 8.S50x Course Team, Copyright 2021 MIT All Rights Reserved.</i>
<hr style="height: 1px;">
<br>

# RECITATION 3: Probability Distributions, Likelihood, and LMFIT software

<br>
<!--end-block--> 

<!--start-block-->
<hr style="height: 1px;">

## 3.0 Overview of Learning Objectives

In this recitation we will explore the following objectives:

- fill in information

<br>
<!--end-block-->

<!--start-block-->
<hr style="height: 1px;">

## 3.1 Fitting Exercise 1

Now, we are going to see how this applies to fitting. Let us first generate some data randomly from from the model $y=2x$.

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Complete the code below to plot the random data and error bars.

<!--
#initial code
import numpy as np
import matplotlib.pyplot as plt
import lmfit.models

np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)
-->

<!--
#solution 
import numpy as np
import matplotlib.pyplot as plt
import lmfit.models

np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)

plt.errorbar(xi, yi, yerr= yerr, fmt = 'o')
plt.show()
-->

<!--end-block-->

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import lmfit.models

np.random.seed(2)

xi = np.array([2,3,4,5,6,7])
yerr = np.array([0.3, 0.4, 0.45, 0.35, 0.6, 0.5])
yi = 2*xi+yerr*np.random.normal(xi.shape)

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Apply a linear fit to the data, plot the fit, and print a report of the fit parameters.

<!--
#initial code
from lmfit.models import LinearModel
model = LinearModel()
-->

<!--
#solution 
from lmfit.models import LinearModel
model = LinearModel()
result = model.fit(yi, x= xi, weights = 1/yerr)

result.plot_fit()
print(result.fit_report())
-->

In [ ]:
from lmfit.models import LinearModel
model = LinearModel()

<!--start-block-->
<hr style="height: 1px;">

## 3.2 Fitting Exercise 2

There are many different types of built-in models in lmfit, including Gaussians, Lorentzians, and polynomials. Let us give some more examples for how to fit different functions. In this next example, use this randomly generated data and fit it to a Gaussian distribution.

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Plot the randomized data, with error bars.


<!--
#initial code
np.random.seed(2)
x = np.linspace(-5 , 5, 50)
yerr = 0.1+np.abs(0.1*np.random.normal(size = x.shape))

y = 5/(np.sqrt(2*np.pi*1.0))*np.exp(-0.5*x**2.0/1.0)+yerr*np.random.normal(size = x.shape)
-->

<!--
#solution 
np.random.seed(2)
x = np.linspace(-5 , 5, 50)
yerr = 0.1+np.abs(0.1*np.random.normal(size = x.shape))

y = 5/(np.sqrt(2*np.pi*1.0))*np.exp(-0.5*x**2.0/1.0)+yerr*np.random.normal(size = x.shape)

plt.errorbar(x, y, yerr= yerr, fmt= 'o')
-->


In [ ]:
np.random.seed(2)
x = np.linspace(-5 , 5, 50)
yerr = 0.1+np.abs(0.1*np.random.normal(size = x.shape))

y = 5/(np.sqrt(2*np.pi*1.0))*np.exp(-0.5*x**2.0/1.0)+yerr*np.random.normal(size = x.shape)

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Apply a Gaussian fit to the data, plot the fit, and print a report of the fit parameters.

<!--
#initial code
from lmfit.models import GaussianModel

gmodel = GaussianModel()

-->

<!--
#solution 
from lmfit.models import GaussianModel

gmodel = GaussianModel()

result = gmodel.fit(y, x=x, weights = 1/yerr)

print(result.fit_report())
result.plot_fit()
-->

In [ ]:
from lmfit.models import GaussianModel

gmodel = GaussianModel()


<!--start-block-->
<hr style="height: 1px;">

## 3.3 Fitting Exercise 3

Note that we can also construct models based on user defined functions. In this case, we need to be careful about the initial values and the limits that we set on the maximum and minimum values.

<!--start-block-->
#### <span style="color:green">>>>RUN</span>

Define a "weird" model by running the cell below.

<!--
#initial code
def weird_model(x, a, b, c):
    return a*np.sin(10*x) + b*np.exp(x)+ c*np.log(x)
-->

In [ ]:
def weird_model(x, a, b, c):
    return a*np.sin(10*x) + b*np.exp(x)+ c*np.log(x)

<!--start-block-->
#### <span style="color:#BA2220">>>>EXERCISE</span>

Plot the randomized data generated by the "weird" model, with error bars.

<!--
#initial code
np.random.seed(2)
x = np.linspace(0.1 , 2, 100)
yerr = 0.5+np.abs(0.5*np.random.normal(size = x.shape))
y = weird_model(x, 2, 0.5, 3)+yerr*np.random.normal(size = x.shape)
-->

<!--
#solution 
np.random.seed(2)
x = np.linspace(0.1 , 2, 100)
yerr = 0.5+np.abs(0.5*np.random.normal(size = x.shape))
y = weird_model(x, 2, 0.5, 3)+yerr*np.random.normal(size = x.shape)

plt.errorbar(x, y, yerr= yerr, fmt= 'o')
-->

In [ ]:
np.random.seed(2)
x = np.linspace(0.1 , 2, 100)
yerr = 0.5+np.abs(0.5*np.random.normal(size = x.shape))
y = weird_model(x, 2, 0.5, 3)+yerr*np.random.normal(size = x.shape)

In [ ]:
from lmfit.models import Model

wmodel = Model(weird_model)

wmodel.set_param_hint(name = 'a', value= 1, min = -10, max =10)
wmodel.set_param_hint(name = 'b', value= 1, min = -10, max =10)
wmodel.set_param_hint(name = 'c', value= 1, min = -10, max =10)

wmodel_result = wmodel.fit(y, x=x, weights = 1/yerr, method = 'dual_annealing')
print(wmodel_result.fit_report())
wmodel_result.plot()

In cases where we have time series data, or we want to evaluate a set of points all at once, it is useful to use the Minimizer class/features of lmfit. In this case, we define a residuals function which is minimized by the lmfit minimize function. This is another way to do the analysis easier.

In [ ]:
from lmfit import Model, minimize, fit_report

def res_weird(params, x, data, error):
    a = params['a']
    b = params['b']
    c = params['c']
    val = a*np.sin(10*x) + b*np.exp(x)+ c*np.log(x)
    return (data-val)/error

In [ ]:
wmodel = Model(weird_model)
params = wmodel.make_params()
params['a'].set(1)
params['b'].set(1)
params['c'].set(1)

res = minimize(res_weird, params = params, args= ( x, y, yerr))

print(fit_report(res))

plt.plot(x, wmodel.eval(params = res.params, x= x ))
plt.errorbar(x, y, yerr= yerr, fmt = '.')

<!--start-block-->
#### <span style="color:purple">>>>QUESTION</span>


Question text needed...


<br>
<!--end-block-->

<!--start-block-->
<hr style="height: 1px;">

## 3.4 Analysis of Time Series Data

Now, we are going to go through an example that is more relevent to the project. Now, this analysis is going to be similar to what you are going to do in your project, but its going to be different in several ways.

In [ ]:
from functions import gwfreq, osc_scale, osc, lower, higher
from gwpy.timeseries import TimeSeries
import h5py
from scipy.interpolate import interp1d

import os

fn = 'H-H1_GWOSC_16KHZ_R1-1126257415-4096.hdf5' # data file
tevent = 1126259462.422 # Mon Sep 14 09:50:45 GMT 2015
evtname = 'GW150914' # event name

detector = 'H1' # detecotr: L1 or H1

strain = TimeSeries.read(fn, format='hdf5.losc')
center = int(tevent)
strain = strain.crop(center-16, center+16)
asd = strain.asd(fftlength = 1)

NRtime, NR_H1 = np.genfromtxt('GW150914_4_NR_waveform.txt').transpose()
nrdata= TimeSeries(NR_H1, times = NRtime)

Define a function that whitens and filters the data.

In [ ]:
def whiten_and_bandpass(strain, asd, bp_low, bp_high):
    fft = strain.fft()
    asdinterp = interp1d(asd.frequencies, asd)
    asddiv =asdinterp(fft.frequencies)
    white_freq = fft/asddiv
    white = white_freq.ifft()
    whitebp = white.bandpass(bp_low,bp_high)
    return TimeSeries(whitebp, t0=strain.t0)

In [ ]:
low_pass = lower()
high_pass = higher()

whitened_data = whiten_and_bandpass(strain, asd, low_pass, high_pass)
wnr = whiten_and_bandpass(nrdata, asd, low_pass, high_pass).crop(-0.1, 0.05)
zoom = whitened_data.crop(tevent-0.09, tevent+0.05)
zoom.t0 = -0.09
x = np.array(zoom.times)
white_data_bp_zoom = zoom.value
plt.plot(x, white_data_bp_zoom)
plt.plot(wnr.times, wnr.value)

Define a funciton that...

In [ ]:
# define osc_dif for lmfit::minimize()
def osc_dif(params, x, data, data_asd, low_pass, high_pass):
    iM=params["Mc"]
    iT0=params["t0"]
    norm=params["C"]
    phi=params["phi"]
    model_strain = TimeSeries(osc(x, iM, iT0, phi, norm), times = x)
    val = whiten_and_bandpass(model_strain, data_asd, low_pass, high_pass)
    residuals = val-data
    return residuals

def osc_white(t, Mc, t0, phi, C, data_asd=None, low_pass = None, high_pass = None):
    model_strain = TimeSeries(osc(t, Mc, t0, phi, C), times = t)
    val = whiten_and_bandpass(model_strain, data_asd, low_pass, high_pass) 
    return val

Plot...

In [ ]:
owmodel = lmfit.Model(osc_white, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
owmodel.set_param_hint(name = 'Mc', value = 25, min = 10, max = 30)
owmodel.set_param_hint(name = 't0', value = 0.004, min = -0.015, max = 0.015)
owmodel.set_param_hint(name = 'C', value = 0.18853556, min = 0.01, max = 10)
owmodel.set_param_hint(name = 'phi', value = 1.24685424, min = 0, max = 2*np.pi)

omodel = lmfit.Model(osc_scale)
omodel.set_param_hint(name = 'Mc', value = 25)
omodel.set_param_hint(name = 't0', value = 0.01)
omodel.set_param_hint(name = 'C', value = 0.18853556)
omodel.set_param_hint(name = 'phi', value = 1.24685424)

result_white = owmodel.fit(white_data_bp_zoom, t = x)
print(result_white.fit_report())
result_white.plot(datafmt='-')

result = omodel.fit(white_data_bp_zoom, t = x)
print(result.fit_report())
result.plot(datafmt='-')



In [ ]:
import lmfit
from lmfit import Model, minimize, fit_report, Parameters

model = lmfit.Model(osc_white)
p = model.make_params()
p['Mc'].set(25)     # Mass guess
p['t0'].set(-0.005)  # By construction we put the merger in the center
p['C'].set(0.18853556)      # normalization guess
p['phi'].set(1.24685424)    # Phase guess
out = minimize(osc_dif, params=p, args=(x, white_data_bp_zoom, asd, low_pass, high_pass))
print(fit_report(out))
best_fit = model.eval(params=out.params,t=x, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
init_fit = model.eval(params=p,t=x, data_asd = asd, low_pass = low_pass, high_pass = high_pass)
plt.plot(x, best_fit,'r',label='best fit')
#plt.plot(x, init_fit,'b',label='init')
plt.plot(x, white_data_bp_zoom)
plt.plot(wnr.times, wnr.value)
plt.show()

<!--start-block-->
#### <span style="color:purple">>>>QUESTION</span>


Question text needed...


<br>
<!--end-block-->